## Categorical VAE para clases
#### Francisco Mena
Basado en:
* https://github.com/EderSantana/gumbel/
* https://github.com/EmilienDupont/vae-concrete

In [12]:
import numpy as np
import keras
from keras.layers import *
from keras.models import Sequential,Model
from keras import backend as K
import tensorflow as tf

#random data
X = np.random.rand(10000,5000)
X.shape

"""
# train the VAE on MNIST digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
"""

"\n# train the VAE on MNIST digits\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\nx_train = x_train.astype('float32') / 255.\nx_test = x_test.astype('float32') / 255.\nx_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))\nx_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))\n"

In [58]:
data_dim = X.shape[1]
Nz = 8 #Number of possible values of discrete variable

tau = K.variable(5.0, name="temperature") #hay que setearlo..

it = Input(shape=(data_dim,))
encoder = Sequential()
encoder.add(Dense(512, activation='relu',input_shape=(data_dim,)))
encoder.add(Dense(256, activation='relu'))
encoder.add(Dense(Nz, activation='linear')) #log(p(z)) o cambiar a : p(z) = softmax() sin log
logits_z  = encoder(it)

def sample_gumbel(shape,eps=K.epsilon()):
    """Inverse Sample function from Gumbel(0, 1)"""
    U = K.random_uniform(shape, 0, 1)
    return - K.log( -K.log(U + eps) + eps)
        
def sampling(logits_z):
    z = logits_z + sample_gumbel(K.shape(logits_z)) # logits + gumbel noise
    return keras.activations.softmax( z/tau )    
    
z_sampled = Lambda(sampling, output_shape=(Nz,))(logits_z)
generator = Sequential()
generator.add(Dense(256, activation='relu', input_shape=(Nz, ))) #a traves de la variable discreta sampleada genera un dato
generator.add(Dense(512, activation='relu'))
generator.add(Dense(data_dim, activation='sigmoid')) #activation, la que requiera el dato
output = generator(z_sampled)


#Me tinca que está mal:
def gumbel_loss(x, x_hat):
    reconstruction_loss =  data_dim * keras.losses.binary_crossentropy(x, x_hat) #loss, la que requiera el problema
    
    #revisar si tiene forma con respecto a los logits:
    num_categories = tuple(logits_z.get_shape().as_list())[1]
    dist =  keras.activations.softmax(logits_z) #para que este normalizado, p(z_i=1)
    dist_sum = K.sum(dist, axis=1)  # Sum over columns, this now has size (batch_size,)
    print(dist_sum) #debiera ser 1...
    dist_neg_entropy = K.sum(dist * K.log(dist + K.epsilon()), axis=1)
    kl_disc_loss =  - ( np.log(num_categories) + K.mean(dist_neg_entropy - dist_sum) )
    
    ##calcular KL con respecto a una entrada auxiliar que es la probabilidad sobre las etiquetas
    #dist =  keras.activations.softmax(logits_z) #para que este normalizado, p(z_i=1)
    #ver si tiene versión con logits
    #keras.losses.kullback_leibler_divergence(y_pred=dist,y_true=entradauxuliaralmodelo)

    
    return reconstruction_loss  + kl_disc_loss #revisar el signo de kl


binary_vae = Model(it, output)
binary_vae.compile(optimizer='adam', loss=gumbel_loss)
binary_vae.summary()

Tensor("loss_7/sequential_20_loss/Sum:0", shape=(?,), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 5000)              0         
_________________________________________________________________
sequential_19 (Sequential)   (None, 8)                 2693896   
_________________________________________________________________
lambda_10 (Lambda)           (None, 8)                 0         
_________________________________________________________________
sequential_20 (Sequential)   (None, 5000)              2698888   
Total params: 5,392,784
Trainable params: 5,392,784
Non-trainable params: 0
_________________________________________________________________


In [52]:
#obtener probabilidades de variable latente categorica (discreta)
logits = tf.convert_to_tensor(encoder.predict(X[:1000]))
probas = keras.activations.softmax(logits).eval(session=K.get_session())
probas

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.9955659e-31,
        0.0000000e+00, 6.3428309e-36],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 8.2947828e-31,
        3.7440332e-38, 1.6957856e-35],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.4488214e-30,
        6.1332934e-38, 2.5001927e-35],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.3322334e-31,
        1.8859822e-38, 8.6474255e-36],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.9190000e-31,
        4.9120987e-38, 1.5498371e-35],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0706194e-30,
        4.8074341e-38, 2.1599924e-35]], dtype=float32)

In [53]:
#generar un nuevo dato dado una variable latente
#dada una probabilidad sobre la variable latente
random_number = tf.convert_to_tensor(np.random.rand(2,Nz))
proba_z = keras.activations.softmax(random_number).eval(session=K.get_session())
z = sampling(proba_z).eval(session=K.get_session())

#dada la variable latente as one-hot
#z = np.zeros((2,Nz))
#z[:,4]=1

reconstruct_data = generator.predict(z)
reconstruct_data

array([[0.5047072 , 0.5001959 , 0.5025172 , ..., 0.49752668, 0.5019674 ,
        0.501997  ],
       [0.50460136, 0.50052494, 0.50289875, ..., 0.49758333, 0.5018671 ,
        0.50213766]], dtype=float32)

In [59]:
batch_size = 100
epochs = 100
binary_vae.fit(X, X,epochs=epochs,batch_size=batch_size)

Epoch 1/100
 8400/10000 [========================>.....] - ETA: 2s - loss: 3464.7220

KeyboardInterrupt: 

In [50]:
X[0]

array([0.19450346, 0.24235764, 0.21172371, ..., 0.19864273, 0.23682403,
       0.66864739])

In [51]:
binary_vae.predict(X)[0]

array([0.50290537, 0.49956557, 0.5020918 , ..., 0.4994231 , 0.5019482 ,
       0.50032127], dtype=float32)